<a href="https://colab.research.google.com/github/aday-94/IU-Spring23-L715NeuralNetworks/blob/main/L715NNS_OpenNMT_mean_rnn_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 63.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  U

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
import torch

torch.cuda.is_available()
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
pwd

'/content'

In [29]:
config = '''#fc-aug-retoke.yaml

## Where the samples will be written
save_data: /content/toke.data
## Where the vocab(s) will be written
src_vocab: /content/toke.vocab.src
tgt_vocab: /content/toke.vocab.tgt
# Prevent overwriting existing files in the folder or allow it
overwrite: True

# Corpus opts:
data:
    train_1:
        path_src: /content/toke-src-1.txt
        path_tgt: /content/toke-tgt-1.txt

    train_2:
        path_src: /content/toke-src-8.txt
        path_tgt: /content/toke-tgt-8.txt

    train_3:
        path_src: /content/toke-tgt-aug-1.txt
        path_tgt: /content/toke-tgt-1.txt

    train_4:
        path_src: /content/toke-tgt-aug-8.txt
        path_tgt: /content/toke-tgt-8.txt

    test:
        path_src: /content/toke-src-5.txt
        path_tgt: /content/toke-tgt-5.txt

# Vocabulary files that were just created
src_vocab: /content/toke.vocab.src
tgt_vocab: /content/toke.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

metrics: accuracy

# Training Model
save_model: /content/mean_rnn_aug/mean_rnn_aug_model
log_file: /content/mean_rnn_aug/mean_rnn_aug_model.log
save_checkpoint_steps: 1000
train_steps: 10000
#valid_steps: 500
#train_epochs:
batch_size: 60
model_task: seq2seq
model_type: text
encoder_type: mean
decoder_type: rnn
rnn_type: LSTM
src_char_input: True

'''

with open("fc-aug-retoke.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat fc-aug-retoke.yaml


#fc-aug-retoke.yaml

## Where the samples will be written
save_data: /content/toke.data
## Where the vocab(s) will be written
src_vocab: /content/toke.vocab.src
tgt_vocab: /content/toke.vocab.tgt
# Prevent overwriting existing files in the folder or allow it
overwrite: True

# Corpus opts:
data:
    train_1:
        path_src: /content/toke-src-1.txt
        path_tgt: /content/toke-tgt-1.txt
    
    train_2:
        path_src: /content/toke-src-8.txt
        path_tgt: /content/toke-tgt-8.txt
    
    train_3:
        path_src: /content/toke-tgt-aug-1.txt
        path_tgt: /content/toke-tgt-1.txt

    train_4:
        path_src: /content/toke-tgt-aug-8.txt
        path_tgt: /content/toke-tgt-8.txt

    test:
        path_src: /content/toke-src-5.txt
        path_tgt: /content/toke-tgt-5.txt

# Vocabulary files that were just created
src_vocab: /content/toke.vocab.src
tgt_vocab: /content/toke.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

metrics: accuracy

# Training Mod

In [30]:
# Build Vocab BRNN_RNN_AUG

!onmt_build_vocab -config fc-aug-retoke.yaml

Corpus train_1's weight should be given. We default it to 1 for you.
Corpus train_2's weight should be given. We default it to 1 for you.
Corpus train_3's weight should be given. We default it to 1 for you.
Corpus train_4's weight should be given. We default it to 1 for you.
Corpus test's weight should be given. We default it to 1 for you.
[2024-05-12 19:59:49,727 INFO] Counter vocab from 5000 samples.
[2024-05-12 19:59:49,727 INFO] Build vocab on 5000 transformed examples/corpus.
[2024-05-12 19:59:50,047 INFO] Counters src: 101
[2024-05-12 19:59:50,047 INFO] Counters tgt: 60
[2024-05-12 19:59:50,047 WARNING] path /content/toke.vocab.src exists, may overwrite...
[2024-05-12 19:59:50,048 WARNING] path /content/toke.vocab.tgt exists, may overwrite...


In [16]:
!mkdir /content/rnn_rnn_aug/

mkdir: cannot create directory ‘/content/rnn_rnn_aug/’: File exists


In [20]:
!mkdir output/

mkdir: cannot create directory ‘output/’: File exists


In [31]:
!onmt_train -config fc-aug-retoke.yaml

Streaming output truncated to the last 5000 lines.
			* train_2: 219
			* train_3: 343
			* train_4: 219
			* test: 554
[2024-05-12 20:09:41,567 INFO] Weighted corpora loaded so far:
			* train_1: 363
			* train_2: 231
			* train_3: 362
			* train_4: 231
			* test: 584
[2024-05-12 20:09:41,568 INFO] Weighted corpora loaded so far:
			* train_1: 363
			* train_2: 231
			* train_3: 363
			* train_4: 231
			* test: 584
[2024-05-12 20:09:41,597 INFO] Weighted corpora loaded so far:
			* train_1: 363
			* train_2: 231
			* train_3: 363
			* train_4: 231
			* test: 585
[2024-05-12 20:09:41,656 INFO] Weighted corpora loaded so far:
			* train_1: 344
			* train_2: 219
			* train_3: 343
			* train_4: 219
			* test: 554
[2024-05-12 20:09:41,661 INFO] Weighted corpora loaded so far:
			* train_1: 344
			* train_2: 219
			* train_3: 344
			* train_4: 219
			* test: 554
[2024-05-12 20:09:41,741 INFO] Weighted corpora loaded so far:
			* train_1: 363
			* train_2: 232
			* train_3: 363
			* train_4:

In [32]:
!onmt_translate -model /content/mean_rnn_aug/mean_rnn_aug_model_step_10000.pt -src /content/toke-src-5.txt -output /content/output/pred_mean_rnn_aug_10000.txt -gpu 0 -verbose

Streaming output truncated to the last 5000 lines.
PRED 266: I n j c · q u j q u a n i ,
PRED SCORE: -0.1104

[2024-05-12 20:13:57,554 INFO] 
SENT 267: ['·', 'i', 'n', 'a', 'q', 'u', 'j', 'n', '·', 't', 'a', 'c', 'h', 'c', 'a', 'u', 'h', '·', ',']
PRED 267: a u h · i n · j c o a c ,
PRED SCORE: -0.1524

[2024-05-12 20:13:57,555 INFO] 
SENT 268: ['·', 'i', 'n', '·', 't', 'a', 'c', 'h', '¶', 'c', 'a', 'u', 'h', 'c', 'h', 'i', 'u', 'h', 't', 'i', 'u', 'h', '·', ',']
PRED 268: c h a l c h i u h t i c ,
PRED SCORE: -0.2925

[2024-05-12 20:13:57,555 INFO] 
SENT 269: ['·', 'i', 'n', 't', 'a', 'c', 'h', 'c', 'a', 'u', 'h', '·', 'i', 'c', 'a', '¶', 't', 'i', 'u', 'h', '·', 'v', 't', 'l', 'i', 'p', 'a', 'n', '·', ',']
PRED 269: a u h · i n · t l a p a c h t l i ,
PRED SCORE: -0.3038

[2024-05-12 20:13:57,555 INFO] 
SENT 270: ['·', 'v', 'n', 'c', 'a', 'n', '·', 'p', 'e', 'o', 'a', '·', ',']
PRED 270: a c o · v n t e p a n ,
PRED SCORE: -0.2520

[2024-05-12 20:13:57,626 INFO] 
SENT 271: ['·', 'i',

In [23]:
!mkdir brnn_rnn_aug

In [28]:
!mkdir mean_rnn_aug